<a href="https://colab.research.google.com/github/dineshkumar-anandan/AWTFrame/blob/new%2Fdev/detectron2_paper_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import gc
import os
import cv2
import torch

import matplotlib.pyplot as plt

from PIL import Image

import detectron2
from detectron2.utils.logger import setup_logger
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode


from yaml import load, FullLoader
from pprint import pprint

/usr/lib/python3.10/imp.py:343: RuntimeWarning: coroutine 'Server.serve' was never awaited
  return _load(spec)


ModuleNotFoundError: ignored

In [ ]:
models = {
    'rpn_r50_fpn': {
        'model_path': 'COCO-Detection/rpn_R_50_FPN_1x.yaml',
        'weights_path': 'model/rpn-r-50-fpn-1x/model_final_02ce48.pkl'
    },
    'faster_rcnn_50': {
        'model_path': 'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml',
        'weights_path': 'model/faster-r-cnn-r50-fpn-x3/model_final_280758.pkl'
    },
    'faster_rcnn_101': {
        'model_path': 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml',
        'weights_path': 'model/detectron2-faster-rcnn-101/model_final_f6e8b1.pkl'
    },
    'retinanet': {
        'model_path': 'COCO-Detection/retinanet_R_101_FPN_3x.yaml',
        'weights_path': 'model/detectron2-faster-rcnn-101/model_final_971ab9.pkl'
    }
}


MODEL_USE = 'faster_rcnn_50'
MODEL_PATH = models[MODEL_USE]['model_path']
WEIGHT_PATH = models[MODEL_USE]['weights_path']

In [ ]:
# Loads the default configuration setup
cfg = get_cfg()
cfg.keys()
cfg['MODEL'].keys()


# pprint(load(open(model_zoo.get_config_file(MODEL_PATH)), Loader=FullLoader))


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(MODEL_PATH))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE='cuda'
cfg.MODEL.WEIGHTS = '/home/dineshkumar.anandan@zucisystems.com/Workspace/models/detectron2_paper_classification_modelv1.pth'
# cfg.MODEL.WEIGHTS = 'detectron2://ImageNetPretrained/MSRA/R-50.pkl'

In [ ]:
predict = DefaultPredictor(cfg)

In [ ]:
%%time

def visual_predict(dataset):
    
    list_final = []
    
    img = cv2.imread(dataset)
    output = predict(img)

    v = Visualizer(img[:, :, ::-1])
    v = v.draw_instance_predictions(output['instances'].to('cpu'))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()
    
    idxofClass = [i for i, x in enumerate(list(output['instances'].pred_classes)) if x == 0]
    
    out = output['instances']
    
    classes = out.pred_classes[idxofClass]
    scores = out.scores[idxofClass]
    boxes = out.pred_boxes[idxofClass]

    list_score = scores.to('cpu').numpy().tolist()
    
    bboxes = boxes.tensor.cpu().numpy().tolist()
    
    count = 1
    
    crop_image = Image.open(dataset)
    
    for score, bbox in zip(list_score, bboxes):
        
        if score > 0.5:
            
            x1, y1, x2, y2 = bbox
            cropped_im = crop_image.crop((x1, y1, x2, y2))
            cropped_im.save(f"cropped_{count}.jpg")
            
            list_final.append({'score':score, 'bbox':bbox})
            
        count+=1
            
    return list_final
                
    
test_dataset = "/home/dineshkumar.anandan@zucisystems.com/detectron2/input/agadia/21.jpg"
visual_predict(test_dataset)